In [ ]:
!python -m spacy download en_core_web_sm

In [113]:
pip install deep-translator 


Note: you may need to restart the kernel to use updated packages.


In [129]:
import pandas as pd
import spacy
from spacy import displacy
import os
import re
import string
import en_core_web_sm
from deep_translator import GoogleTranslator 
nlp = en_core_web_sm.load()

In [99]:
JobSkills = pd.read_excel(r"..\NLP\ResumeSkill_preprocessed.xlsx")
# lower characters and add spaces to the side to spot the complete words
# DE_JobSkills = (" " + JobSkills["Data_Engineer"].dropna().str.lower() + " ").to_list()
DE_JobSkills = JobSkills["Data_Engineer"].dropna().str.lower().to_list()
DE_JobSkills[0:10]



['api',
 'rest',
 'shell',
 'cli',
 'cronjobs',
 'python',
 'java',
 'scala',
 'go',
 'sql']

In [86]:
JSON_FILEPATH = r"..\Bumeran\bumeran_scraped_data_22-06-2022_1129.json"
json_df = pd.read_json(JSON_FILEPATH, orient='records')

In [87]:
json_df.head()

,keyWords,title_role,description,date_published,schema,consulted_datetime,source,company_rate,salary,company,city
0,"[ingeniero de datos, data engineer]",Analytics Data Engineer,Somos una empresa 100% mexicana con más de 25 ...,Publicado hace más de 30 días,Presencial,"22/06/2022, 11:29",https://www.bumeran.com.mx/empleos/analytics-d...,NaN,No especificado,Human Quality,"Cuajimalpa de Morelos, Distrito Federal, Mexico"
1,"[ingeniero de datos, data engineer]",Data Engineer with Azure and Analytics,Somos una empresa 100% mexicana con más de 25 ...,Publicado hace más de 30 días,Presencial,"22/06/2022, 11:29",https://www.bumeran.com.mx/empleos/data-engine...,NaN,No especificado,Human Quality,"Cuajimalpa de Morelos, Distrito Federal, Mexico"


In [88]:

rows_containing_salary = json_df.loc[json_df["description"].str.contains("Salario")]

In [120]:
json_df['blob_en'] = None

In [130]:
text = json_df['description'][0] 

if len(text) >= 5000: 

    # sentences = text.split(sep = '.') 

    text = text.replace('\n', ' ') 

    sentences = re.split(r'(?<=\D)\.(?=.)|(?<=\d)\.(?=\D)', text) 

    # filter_object = filter(lambda x: x != "", sentences) 

    filter_object = filter(lambda x: len(x) > 2, sentences) 

    sentences = list(filter_object) 

    filter_object = filter(lambda x: bool(re.search('[a-zA-Z]', x)), sentences)  

    sentences = list(filter_object) 

    filter_object = filter(lambda x: (sum(c.isalpha() for c in x)) > 2, sentences) 

    sentences = list(filter_object) 

    translated = GoogleTranslator('es', 'en').translate_batch(sentences) 

    translated_news = '. '.join(translated) 

    json_df['blob_en'][0] = translated_news 

else: 

    translated_news = GoogleTranslator(source='es', target='en').translate(text) 

    json_df['blob_en'][0] = translated_news 

C:\Users\vandarcia\AppData\Local\Temp\ipykernel_18356\4095072707.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['blob_en'][0] = translated_news


In [124]:
SAMPLE_TEXT = json_df['blob_en'][0]
SAMPLE_TEXT


"We are a 100% Mexican company with more than 25 years of experience in connecting talent with companies.\n\n\nAnalytics Data Engineer in Monterrey, Guadalajara and AguasCalientes\nThe ideal candidate is a motivated individual who is excited about making an impact to a user base of over 20k that is ready to operate in a fast-paced environment and can clearly demonstrate a thirst for learning and mastering emerging technologies, processes and tools in the analyticsspace. This person should take initiative and shouldn't be an order taker. They should enjoy optimizing data models to industry standards.\n\n\nKey Requirements:\n\n1. Enterprise level SSAS / Tabular DAX Modeling experience\n2. Experience with SQL Querying\n3. Experienced with agile processes and methodologies\n4. Experience with Production Models & troubleshooting skills\n5. Ability to demonstrate understanding of business requirements & business concepts\n\nFollowing skills are not required but are a plus :\n\n6. Experience 

In [125]:
# Data Cleaning

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    # text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub(r'[^\w\s$]', ' ', text) 
    # text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    # text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text
 
 
data_clean = clean_text_round1(SAMPLE_TEXT)
 

data_clean

'we are a 100  mexican company with more than 25 years of experience in connecting talent with companies    analytics data engineer in monterrey  guadalajara and aguascalientes the ideal candidate is a motivated individual who is excited about making an impact to a user base of over 20k that is ready to operate in a fast paced environment and can clearly demonstrate a thirst for learning and mastering emerging technologies  processes and tools in the analyticsspace  this person should take initiative and shouldn t be an order taker  they should enjoy optimizing data models to industry standards    key requirements   1  enterprise level ssas   tabular dax modeling experience 2  experience with sql querying 3  experienced with agile processes and methodologies 4  experience with production models   troubleshooting skills 5  ability to demonstrate understanding of business requirements   business concepts  following skills are not required but are a plus    6  experience working with a te

In [128]:
# { jobSkill:[x.span() for x in re.finditer(jobSkill,SAMPLE_TEXT.lower())]  for jobSkill in DE_JobSkills} 
JobSkillsLoc = { jobSkill:[x.span() for x in re.finditer(" "+ jobSkill + " ",data_clean)] for jobSkill in DE_JobSkills} 
JobSkillsLoc

{'api': [],
 'rest': [],
 'shell': [],
 'cli': [],
 'cronjobs': [],
 'python': [],
 'java': [],
 'scala': [],
 'go': [],
 'sql': [(700, 705)],
 'acid': [],
 'cap': [],
 'oltp': [],
 'olap': [],
 'mysql': [],
 'postgresql': [],
 'mariadb': [],
 'amazon aurora': [],
 'mongodb': [],
 'elasticsearch': [],
 'apache couchdb': [],
 'azure cosmosdb': [],
 'apache cassandra': [],
 'apache hbase': [],
 'google bigtable': [],
 'neo4j': [],
 'amazon neptune': [],
 'redis': [],
 'memcached': [],
 'amazon dynamodb': [],
 'snowflake': [],
 'presto': [],
 'apache hive': [],
 'apache impala': [],
 'amazon redshift': [],
 'azure synapse': [],
 'clickhouse': [],
 'apache hadoop': [],
 'hdfs': [],
 'mapreduce': [],
 'amazon mr': [],
 'google dataproc': [],
 'azure data lake': [],
 'apache pig': [],
 'apache arrow': [],
 'data build tool': [],
 'apache spark': [],
 'apache beam': [],
 'apache flink': [],
 'apache nifi': [],
 'apache kafka': [],
 'apache storm': [],
 'apache samza': [],
 'amazon kinesis': [

In [107]:
# nlp = spacy.load("es_core_news_sm")
nlp = en_core_web_sm.load()


# skill = "XPATH" 
# [skill[:],"Skill"]


# {"SKILL",""}
# Vales de despensa, "Beneficios"

In [111]:
doc = nlp(rows_containing_salary["description"].iloc[0])

In [112]:
for ent in doc.ents:
    print(ent.text,ent.label_)

100% PERCENT
de experiencia PERSON
con ORG
Analytics Data Engineer ORG
Monterrey GPE
Guadalajara GPE
20k DATE
SSAS / ORG
2 CARDINAL
SQL Querying ORG
3 CARDINAL
4 CARDINAL
Production Models ORG
5 CARDINAL
Git ORG
7 CARDINAL
Spark/Databricks Experience ORG
8 CARDINAL
English LANGUAGE
MXN  

 ORG
one CARDINAL
3 CARDINAL
Guadalajara GPE
Monterrey PERSON
English LANGUAGE
Cuajimalpa de Morelos PERSON
Distrito Federal ORG
apta para personas PERSON
El proceso de selección ORG
diseñado para PERSON
tu experiencia de PERSON
1 CARDINAL
2 CARDINAL
Revisar tu mail PERSON
3 CARDINAL
vemos que PERSON
tu perfil PERSON
estamos NORP
la GPE
